In [2]:
from datetime import datetime,timedelta
from collections import defaultdict,Counter
from pprint import pprint
from tqdm import tqdm
import re

import pymongo
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
from pymongo.errors import BulkWriteError

In [3]:
client = pymongo.MongoClient('localhost:27017')
db = client.tweet

In [4]:
import pandas as pd
import spacy

In [13]:
nlp = spacy.load('en_core_web_md')

In [108]:
events = [e for e in db.current_event.find({'event.date':{'$gt':'2010-09-01','$lt':'2010-09-03'}},{'_id':1,'abstracts':1,'type':1})]

In [109]:
for i in events:
    print(i)

{'_id': ObjectId('59fbc9fa60b18848c5a4d4e4'), 'type': 1, 'abstracts': [{'media': ['BBC', 'Reuters'], 'abstract': '10 civilians are killed and 2 others are wounded after being struck by NATO during an election campaign in Rostaq, Afghanistan. Originally, a spokesman had said a "precision air strike" had hit a militant vehicle.'}]}
{'_id': ObjectId('59fbc9fa60b18848c5a4d4e5'), 'type': 1, 'abstracts': [{'media': ['YNet News'], 'abstract': 'Palestinians stone an Israeli car travelling through the West Bank resulting in a 12-year-old being injured.'}]}
{'_id': ObjectId('59fbc9fa60b18848c5a4d4e6'), 'type': 1, 'abstracts': [{'media': ['BBC'], 'abstract': 'The Mexican Army claims to have killed 25 drug cartel gunmen in a clash in Tamaulipas state near the United States border.'}]}
{'_id': ObjectId('59fbc9fa60b18848c5a4d4e7'), 'type': 2, 'abstracts': [{'media': ['BBC'], 'abstract': 'Permanent Representative of Pakistan to the United Nations Hussain Haroon calls for an inquiry after allegations 

In [110]:
events[0]

{'_id': ObjectId('59fbc9fa60b18848c5a4d4e4'),
 'abstracts': [{'abstract': '10 civilians are killed and 2 others are wounded after being struck by NATO during an election campaign in Rostaq, Afghanistan. Originally, a spokesman had said a "precision air strike" had hit a militant vehicle.',
   'media': ['BBC', 'Reuters']}],
 'type': 1}

In [111]:
abst

'Seven people are killed and 13 are injured after suicide bombers hit a state-run power station near the northern city of Samarra, Iraq. The Islamic State of Iraq and the Levant claims responsibility for the attack.'

In [113]:
records

'903363851781099522'

In [125]:
def reference_similatity(event):
    abst = ''.join([i['abstract'] for i in event['abstracts']])
    doc_reference = nlp(abst)
    print(abst)
    tweets = []
    #filter_dict = {'event_id':events[0]['_id'],'tweet.lang':'en','tweet.media.card_url':{'$ne':None}}
    #query_dict = {'_id':1}
    #first_news_tweet_id  = min([i['_id'] for i in db.pos.find(filter_dict,query_dict).sort('_id').limit(1)]+[i['_id'] for i in db.paper.find(filter_dict,query_dict).sort('_id').limit(1)])
    filter_dict = {'event_id':event['_id'],'tweet.lang':'en','tweet.media.card_url':None}
    query_dict = {'tweet.standard_text':1}
    records  = [i for i in db.pos.find(filter_dict,query_dict)]+[i for i in db.paper.find(filter_dict,query_dict)]
    print(len(records))
    for tweet in records:
        tweet_id = tweet['_id']
        tweet_text = tweet['tweet']['standard_text'] 
        doc_tweet = nlp(tweet_text)
        tweets.append((tweet_id,doc_reference.similarity(doc_tweet),tweet_text))
    tweets = sorted(tweets,key=lambda x:x[1],reverse=True)
    df_candidate = pd.DataFrame.from_records(tweets,index=range(len(tweets)),columns=['id','simi','text'])
    df_candidate = df_candidate.drop_duplicates(['simi'])
    return df_candidate[df_candidate['simi'] > 0.75][:100]

In [128]:
'\n'.join(['a','b'])

'a\nb'

In [126]:
x[:3]

,id,simi,text
0,22805330372,0.962651,The three Pakistan cricketers accused of corru...
1,22840215879,0.946432,The International Cricket Council has suspende...
2,22838266153,0.942557,All three Pakistan cricketers accused of corru...


In [127]:
for i in events:
    x = reference_similatity(i)
    print(len(x))

10 civilians are killed and 2 others are wounded after being struck by NATO during an election campaign in Rostaq, Afghanistan. Originally, a spokesman had said a "precision air strike" had hit a militant vehicle.
155
100
Palestinians stone an Israeli car travelling through the West Bank resulting in a 12-year-old being injured.
31
19
The Mexican Army claims to have killed 25 drug cartel gunmen in a clash in Tamaulipas state near the United States border.
329
100
Permanent Representative of Pakistan to the United Nations Hussain Haroon calls for an inquiry after allegations emerge of Pakistani floodwaters being diverted into vulnerable villages in a bid to preserve crops.
183
100
Severe Tropical Storm Lionrock lands in China's Fujian province with warnings of strong winds and torrential rains.Typhoon Kompasu (Glenda) hits South Korea resulting in three deaths and leading to cancellation of flights and school classes.
143
80
A tropical storm warning is issued for the coast of Long Islan